# Random Walk with Restart using MultiXrank

This code runs MultiXrank on a network composed of 2 layers:
- RARE-X diseases - patients - symtomes associations network
- ORPHANET diseases - phenotypes associations network

We use the 27 diseases from the RARE-X layer as seeds for the RWR

In [1]:
import multixrank
import pandas as pd
import glob
import os

In [2]:
def create_seeds_file(diseases_names_files: str, path_to_seeds_files: str) -> None:
    diseases = pd.read_csv(diseases_names_files, sep=";", header=0)
    i = 0
    j = 1
    for index, row in diseases.iterrows():
        seeds_file= path_to_seeds_files + "/" + f"seeds_{j}.txt"
        seeds = pd.DataFrame(columns=["seed"])
        if row[1] == "None":
            seeds._set_value(i, "seed", row[0])
        else:
            seeds._set_value(i, "seed", row[1])
        seeds.to_csv(seeds_file, sep="\t", header=None, index=False)
        i += 1
        j += 1
    return(j)

seedNb = create_seeds_file(diseases_names_files="../Diseases_Rx_orpha_corres.csv", path_to_seeds_files="../multixrank_RARE_X_diseases/seeds_files/")    

In [3]:
def create_config_files(path_to_config_files: str) -> None:
    layers_1 = ['../network/multiplex/RARE_X/RARE_X_layer.tsv']
    layers_2 = ['../network/multiplex/Orpha/Disease_Phenotype.tsv']
    size = 1
    for i in range(1, seedNb):
        file = open(path_to_config_files + f'/config_{i}.yml', 'w')
        r = 0.7
        delta = 0
        eta = [1, 0]
        tau = [1]

        file.write('seed:' + ' ' + path_to_config_files + f'../seeds_files/seeds_{i}.txt' + '\n')
        file.write('self_loops: 0' + '\n')
        file.write('r: ' + str(r) + '\n')
        temp = '{},'*size
        part = '[' + temp.rstrip(',') +']'
        file.write('eta: ' + str(eta) + '\n')
        file.write('lamb:' + '\n')
        file.write('    ' + '-' +  ' ' + '[' + str(1/2) + ',' + str(1/2) + ']' + '\n')
        file.write('    ' + '-' +  ' ' + '[' + str(1/2) + ',' + str(1/2) + ']' + '\n')
        file.write('multiplex:' + '\n')
        
        file.write('    ' + 'Rare_X_layer' + ':' + '\n' + '        ' + \
                        'layers:' + '\n' + '            ')
        file.write('-' +  ' ' + layers_1[0] + '\n' + '        ')
        
        file.write('delta: {}'.format(str(delta)) + '\n' + '        ' )
        file.write('graph_type: ' + '[' + ('00, '*size).rstrip(', ') + ']' + '\n' + '        ' )
        file.write('tau: ' + str(tau) + '\n')
        
        file.write('    ' + 'Orpha_layer' + ':' + '\n' + '        ' + \
                        'layers:' + '\n' + '            ')
        file.write('-' + ' ' + layers_2[0] + '\n' + '        ')
        file.write('delta: {}'.format(str(delta)) + '\n' + '        ' )
        file.write('graph_type: ' + '[' + ('01, '*size).rstrip(', ') + ']' + '\n' + '        ' )
        file.write('tau: ' + str(tau) + '\n')

        file.write('bipartite:' + '\n')
        file.write("    " +  "../network/bipartite/bipartite_RARE_X_orpha_diseases.tsv: {'source': 'Rare_X_layer', 'target': 'Orpha_layer', graph_type: 00}" + '\n')
        file.close

create_config_files(path_to_config_files="../multixrank_RARE_X_diseases/config_files/")

In [4]:
for i in range(1, seedNb):
    multixrank_obj = multixrank.Multixrank(config=f"../multixrank_RARE_X_diseases/config_files/config_{i}.yml", wdir="../multixrank_RARE_X_diseases/")
    ranking_df = multixrank_obj.random_walk_rank()
    os.mkdir(f"../multixrank_RARE_X_diseases/output_multixrank/output_{i}")
    multixrank_obj.write_ranking(ranking_df, path=f"../multixrank_RARE_X_diseases/output_multixrank/output_{i}/")